In [17]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from src.inference import get_feature_store

In [3]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 00:18:22,890 INFO: Initializing external client
2025-03-06 00:18:22,890 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:18:23,707 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691
Fetching data from 2025-02-05 05:18:22.890346+00:00 to 2025-03-06 04:18:22.890346+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.84s) 


In [4]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 06:00:00+00:00,2,0
1,2025-02-05 07:00:00+00:00,2,0
2,2025-02-05 08:00:00+00:00,2,0
3,2025-02-05 09:00:00+00:00,2,0
4,2025-02-05 10:00:00+00:00,2,0
...,...,...,...
174631,2025-03-05 22:00:00+00:00,263,82
174632,2025-03-05 23:00:00+00:00,263,28
174633,2025-03-06 00:00:00+00:00,263,16
174634,2025-03-06 01:00:00+00:00,263,6


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174636 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174636 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174636 non-null  int32                  
 2   rides               174636 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [6]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [7]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174636 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174636 non-null  datetime64[us]
 1   pickup_location_id  174636 non-null  int32         
 2   rides               174636 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [8]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [9]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,155,80,14,106,16,129,14,42,5,...,87,88,109,7,17,88,112,27,263,2025-02-23 14:00:00
1,2,3,0,0,1,0,17,0,13,2,...,20,0,1,1,1,1,1,0,255,2025-02-22 16:00:00
2,0,0,0,0,1,0,0,2,0,0,...,3,0,1,0,1,0,0,1,69,2025-02-22 16:00:00
3,1,0,1,2,0,0,0,0,0,0,...,1,0,2,0,0,0,1,0,155,2025-02-23 03:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,171,2025-03-02 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,173,2025-02-10 23:00:00
248,1,0,1,1,0,0,1,1,1,0,...,0,0,0,1,2,0,1,0,157,2025-03-04 21:00:00
249,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,94,2025-02-24 22:00:00
250,2,8,3,1,1,0,3,0,1,0,...,1,3,0,1,10,1,5,0,97,2025-03-05 07:00:00


In [10]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 00:18:33,598 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:18:33,605 INFO: Initializing external client
2025-03-06 00:18:33,606 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:18:34,314 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691
Fetching data from 2025-02-05 05:18:33.598679+00:00 to 2025-03-06 04:18:33.598679+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.36s) 


In [11]:
current_date

Timestamp('2025-03-06 05:18:33.598679+0000', tz='Etc/UTC')

In [12]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 06:00:00
1,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,3,2025-03-05 06:00:00
2,0,4,6,2,3,2,0,1,3,2,...,3,2,5,3,3,0,0,0,4,2025-03-05 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-05 06:00:00
4,2,6,4,2,1,4,1,2,1,2,...,0,1,1,0,2,0,2,0,7,2025-03-05 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-05 06:00:00
248,0,1,4,1,0,3,2,5,0,1,...,0,1,1,0,0,0,1,2,260,2025-03-05 06:00:00
249,7,8,15,14,12,23,23,25,28,23,...,22,13,4,1,0,0,0,2,261,2025-03-05 06:00:00
250,66,151,147,158,114,84,84,65,73,99,...,10,5,1,2,2,4,7,18,262,2025-03-05 06:00:00


In [13]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 00:18:44,878 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 00:18:44,884 INFO: Initializing external client
2025-03-06 00:18:44,884 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:18:45,631 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691


In [14]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\2003j\anaconda3\envs\taxi_env\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [15]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,2.0
...,...,...
247,259,1.0
248,260,1.0
249,261,4.0
250,262,75.0


In [16]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([186, 100, 132, 162, 262, 236, 238, 263,  48, 239])